In [ ]:
import os
import re

import numpy as np
import pandas as pd
from scipy.optimize import minimize_scalar

import seaborn as sns
import matplotlib.pylab as plt

from luescher_nd.database import utilities as ut
from luescher_nd.database.utilities import DATA_FOLDER
from luescher_nd.zeta.extern.pyzeta import zeta

from luescher_nd.plotting import styles

styles.setup(pgf=False)

In [ ]:
%load_ext blackcellmagic

In [ ]:
a_inv = 0.0
L = 1.0

In [ ]:
diff_sq = lambda x: (a_inv - zeta(x)[0] / np.pi / L) ** 2
bounds = [-10] + [n2 for n2 in ut.get_degeneracy(20) if n2 < 50]
xs = []

for b1, b2 in zip(bounds, np.roll(bounds, -1)):
    if b1 > b2: break
    
    xs.append(
        minimize_scalar(
            diff_sq,
            method="bounded",
            bounds=(b1 + 1.0e-3, b2 - 1.0e-3),
            options={"xatol": 1.0e-16}
        ).x
    )

spectrum = np.array(xs)

In [ ]:
files = [f for f in os.listdir(DATA_FOLDER) if f.endswith(".sqlite") and not "tmp" in f]

In [ ]:
file_name = f"contact-fitted_a-inv={a_inv:+1.1f}_zeta=spherical_projector=a1g_n-eigs=200.sqlite"
df = ut.read_table(
    os.path.join(DATA_FOLDER, file_name),
    zeta=None,
    round_digits=2,
    filter_poles=False,
    filter_by_nstates=False,
    filter_degeneracy=False,
).query("nlevel < 24 and epsilon < 0.2 and L == @L")[
    ["n1d", "epsilon", "nstep", "L", "x", "nlevel", "mass"]
]
df["L"] = df.L.round(7)
df.head()

In [ ]:
data = []
for idx, (l, nstep) in df[["L", "nstep"]].drop_duplicates().iterrows():
    for nlevel, x in enumerate(spectrum):
        data.append({
            "L": l,
            "epsilon": 0,
            "nstep": int(nstep),
            "n1d": None,
            "x": x,
            "nlevel": nlevel,
        })
tf = pd.DataFrame(data)

for deg in ut.get_degeneracy_list(20):
    tf.loc[tf.nlevel >= deg, "nlevel"] += 1

In [ ]:
ff = df.groupby(["n1d", "epsilon"]).apply(
    lambda frame: (frame.set_index(["L", "nstep", "nlevel"])[["x"]]
    - tf.set_index(["L", "nstep", "nlevel"])[["x"]]).abs()
).reset_index().dropna()

ff["diff_e"] = ff["x"] / ff["epsilon"] / (df["mass"].unique()[0]/2)
ff["e2"] = ff["epsilon"]**2

ff["nstep_label"] = ff.nstep.where(ff.nstep > 0, "$\infty$")

ff.head()

In [ ]:
grid = sns.FacetGrid(
    data=ff.sort_values("epsilon").query("nlevel > 0 and nlevel < 5"),
    col="nlevel",
    hue="nstep_label",
    col_wrap=4,
    sharey=True,
    margin_titles=True,
    hue_order=[1,2,4,r"$\infty$"]
)

grid.map(plt.plot, "epsilon", "diff_e", marker=".", ls=":", zorder=10)
grid.add_legend(title="$n_\mathrm{step}$")

for ax in grid.axes.flat:
    ax.set_yscale("log")
    ax.set_xscale("log")
    ax.set_xlim(1.9e-2, 2**-4)
    
grid.set_xlabels("$\epsilon \, [\mathrm{fm}]$")
grid.set_ylabels(r"$\left|x_A - x_N\right| / (\mu\epsilon)$")

styles.finalize(grid.fig, width=None)

In [ ]:
grid.savefig("continuum-diff-detail.jpg", bbox_inches="tight")

In [ ]:
ff["even"] = ff.n1d % 2 == 0

grid = sns.FacetGrid(
    data=ff.query("nlevel > 0 and nlevel < 2").query("epsilon < 0.05"),
    col="nstep",
    row="nlevel",
    hue="even",
    sharey=False,
    margin_titles=True,
    col_order=[1,2,4,-1]
)

grid.map(plt.plot, "epsilon", "diff_e", marker="o", ls=":", zorder=10)
grid.add_legend(title="$n_{1d}$ even")

for ax in grid.axes.flat:
    ax.set_yscale("log")
    ax.set_xscale("log")

grid.set_xlabels("$\epsilon \, [\mathrm{fm}]$")
grid.set_ylabels(r"$\left|x_A - x_N\right| / (\mu\epsilon)$")

styles.finalize(grid.fig, width=None)

In [ ]:
grid.savefig("continuum-diff-detail.jpg", bbox_inches="tight")